In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import sys
sys.path.append('../')

In [118]:
from src.utils import loading, Spark
import pyspark.ml as M
import pyspark.sql.functions as F
import pyspark.sql.types as T
import numpy as np
import math

In [4]:
spark = Spark()

Spark UI address http://127.0.0.1:4040


In [5]:
datas = loading(spark, '../data/raw')

In [6]:
data = datas['sample']

In [41]:
data.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|120380|   1020|   3.0|
|121430|   1020|   3.0|
|141478|   1020|   4.0|
|178042|   1020|   4.0|
|179023|   1020|   2.0|
|181736|   1020|   3.5|
|195602|   1020|   3.5|
|  2160|   1020|   4.5|
| 57551|   1020|   3.0|
|153188|   1020|   4.0|
|246356|   1020|   1.5|
|274232|   1020|   3.0|
| 54291|   1020|   2.5|
| 90252|   1020|   2.5|
|242983|   1020|   3.0|
|280429|   1020|   3.0|
| 83288|   1020|   2.0|
|155987|   1020|   3.0|
|168976|   1020|   3.5|
|227402|   1020|   1.5|
+------+-------+------+
only showing top 20 rows



In [217]:
def create_rating_matrix(data):
    df = data.withColumn("rating", data.rating.cast('float'))
    return df.groupby("userId").pivot("movieId").sum("rating")

def compute_r(df, u, filmId, k):
    sims = []
    svs = []
    
    users_unique = []
    users = df.select("userId", filmId).collect()
    for i in users:
        if i[1] != None:
            users_unique.append(i[0])
    
    row_u = np.array(list(df.filter(df.userId == u).collect()[0][1:]))
    u_num = row_u[row_u != None]
    muu = u_num.mean()
        
    for v in users_unique:
        row_v = np.array(list(df.filter(df.userId == v).collect()[0][1:]))
        v_num = row_v[row_v != None]
        muv = v_num.mean()
        
        svs.append(df.filter(df.userId == v).collect()[0][filmId] - muv)
    
        intersection = []
        for i in range(len(row_u)):
            if row_u[i] != None and row_v[i] != None:
                intersection.append(i)
                
        if len(intersection) == 0:
            break
    
        u_inter = np.array(row_u[intersection])
        v_inter = np.array(row_v[intersection])
    
        numerator = (u_inter - muu).dot(v_inter - muv)
        denumerator = math.sqrt(np.sum((u_inter - muu)**2)) * math.sqrt(np.sum((v_inter - muv)**2))
        sims.append(numerator/denumerator)
        
    svs = np.array(svs)
    sims = np.array(sims)
    
    idx = (-sims).argsort()[:k]
    
    svs = svs[idx]
    sims = sims[idx]
    
    if len(sims) == 0:
        return muu
    
    numerator = sims.dot(svs)
    denumerator = np.sum(abs(sims))
    
    return muu + numerator/denumerator
    